In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import copy
from stocktrends import Renko
import statsmodels.api as sm

def MACD(DF,a,b,c):
    df = DF.copy()
    df["MA_Fast"]=df["Adj Close"].ewm(span=a,min_periods=a).mean()
    df["MA_Slow"]=df["Adj Close"].ewm(span=b,min_periods=b).mean()
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"]
    df["Signal"]=df["MACD"].ewm(span=c,min_periods=c).mean()
    df.dropna(inplace=True)
    return (df["MACD"],df["Signal"])

def OBV(DF):
    df = DF.copy()
    df['daily_ret'] = df['Adj Close'].pct_change()
    df['direction'] = np.where(df['daily_ret']>=0,1,-1)
    df['direction'][0] = 0
    df['vol_adj'] = df['Volume'] * df['direction']
    df['obv'] = df['vol_adj'].cumsum()
    return df['obv']


def ATR(DF,n):
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2

def slope(ser,n):
    slopes = [i*0 for i in range(n-1)]
    for i in range(n,len(ser)+1):
        y = ser[i-n:i]
        x = np.array(range(n))
        y_scaled = (y - y.min())/(y.max() - y.min())
        x_scaled = (x - x.min())/(x.max() - x.min())
        x_scaled = sm.add_constant(x_scaled)
        model = sm.OLS(y_scaled,x_scaled)
        results = model.fit()
        slopes.append(results.params[-1])
    slope_angle = (np.rad2deg(np.arctan(np.array(slopes))))
    return np.array(slope_angle)

'''
def renko_DF(DF):
    df = DF.copy()
    df.reset_index(inplace=True)
    df = df.iloc[:,[0,1,2,3,4,5]]
    df.columns = ["date","open","high","low","close","volume"]
    df2 = Renko(df)
    df2.brick_size = max(0.5,round(ATR(DF,120)["ATR"][-1],0))
    renko_df = df2.get_ohlc_data()
    renko_df["bar_num"] = np.where(renko_df["uptrend"]==True,1,np.where(renko_df["uptrend"]==False,-1,0))
    for i in range(1,len(renko_df["bar_num"])):
        if renko_df["bar_num"][i]>0 and renko_df["bar_num"][i-1]>0:
            renko_df["bar_num"][i]+=renko_df["bar_num"][i-1]
        elif renko_df["bar_num"][i]<0 and renko_df["bar_num"][i-1]<0:
            renko_df["bar_num"][i]+=renko_df["bar_num"][i-1]
    renko_df.drop_duplicates(subset="date",keep="last",inplace=True)
    return renko_df
'''

def CAGR(DF):
    df = DF.copy()
    df["cum_return"] = (1 + df["ret"]).cumprod()
    n = len(df)/(252*78)
    CAGR = (df["cum_return"].tolist()[-1])**(1/n) - 1
    return CAGR

def volatility(DF):
    df = DF.copy()
    vol = df["ret"].std() * np.sqrt(252*78)
    return vol

def sharpe(DF,rf):
    df = DF.copy()
    sr = (CAGR(df) - rf)/volatility(df)
    return sr
    

def max_dd(DF):
    df = DF.copy()
    df["cum_return"] = (1 + df["ret"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd

# Download historical data for DJI constituent stocks

tickers = ["RELIANCE.NS","ASIANPAINT.NS","TCS.NS","ICICIBANK.NS","EICHERMOT.NS"]


ohlc_intraday = {}           

for i in range(len(tickers)):
    ohlc_intraday[tickers[i]] = yf.download(tickers[i], interval = "5m", start = "2020-08-17", end = "2020-09-15")
    ohlc_intraday[tickers[i]].drop(["Close"],axis = 1, inplace = True)
    ohlc_intraday[tickers[i]].columns = ["Open","High","Low","Adj Close","Volume"]     

################################Backtesting####################################


#Merging renko df with original ohlc df
ohlc_merged = {}
df = copy.deepcopy(ohlc_intraday)
tickers_signal = {}
tickers_ret = {}
for ticker in tickers:
    print("merging for ",ticker)
    ohlc_merged[ticker]["macd"]= MACD(ohlc_merged[ticker],12,26,9)[0]
    ohlc_merged[ticker]["macd_sig"]= MACD(ohlc_merged[ticker],12,26,9)[1]
    ohlc_merged[ticker]["macd_slope"] = slope(ohlc_merged[ticker]["macd"],5)
    ohlc_merged[ticker]ohlc_merged[ticker]["macd_sig_slope"] = slope(ohlc_merged[ticker]["macd_sig"],5)
    ohlc_merged[ticker]["obv"]= OBV(ohlc_merged[ticker])
    ohlc_merged[ticker]["obv_slope"]= slope(ohlc_merged[ticker]["obv"],5)
    tickers_signal[ticker] = ""
    tickers_ret[ticker] = []
 


#Identifying signals and calculating daily return
for ticker in tickers:
    print("calculating daily returns for ",ticker)
    for i in range(len(ohlc_intraday[ticker])):
        if tickers_signal[ticker] == "":
            tickers_ret[ticker].append(0)
            if i > 0:
                if ohlc_merged[ticker]["obv_slope"][i]>30 and ohlc_merged[ticker]["macd"][i]>ohlc_merged[ticker]["macd_sig"][i] and ohlc_merged[ticker]["macd_slope"][i]>ohlc_merged[ticker]["macd_sig_slope"][i]:
                    tickers_signal[ticker] = "Buy"
                
        
        elif tickers_signal[ticker] == "Buy":
            tickers_ret[ticker].append((ohlc_merged[ticker]["Adj Close"][i]/ohlc_merged[ticker]["Adj Close"][i-1])-1)
            if i > 0:
                if ohlc_merged[ticker]["macd"][i]<ohlc_merged[ticker]["macd_sig"][i] and ohlc_merged[ticker]["macd_slope"][i]<ohlc_merged[ticker]["macd_sig_slope"][i]:
                    tickers_signal[ticker] = ""
                
        
    ohlc_merged[ticker]["ret"] = np.array(tickers_ret[ticker])

#calculating overall strategy's KPIs
strategy_df = pd.DataFrame()
for ticker in tickers:
    strategy_df[ticker] = ohlc_merged[ticker]["ret"]
strategy_df["ret"] = strategy_df.mean(axis=1)
CAGR(strategy_df)
sharpe(strategy_df,0.025)
max_dd(strategy_df)  
 
#visualizing strategy returns
(1+strategy_df["ret"]).cumprod().plot()
 
#calculating individual stock's KPIs
cagr = {}
sharpe_ratios = {}
max_drawdown = {}
for ticker in tickers:
    print("calculating KPIs for ",ticker)      
    cagr[ticker] =  CAGR(ohlc_merged[ticker])
    sharpe_ratios[ticker] =  sharpe(ohlc_merged[ticker],0.025)
    max_drawdown[ticker] =  max_dd(ohlc_merged[ticker])
 

KPI_df = pd.DataFrame([cagr,sharpe_ratios,max_drawdown],index=["Return","Sharpe Ratio","Max Drawdown"])      
KPI_df.T

SyntaxError: invalid syntax (<ipython-input-2-8e2d56cd0890>, line 122)

In [ ]:
tickers_signal